In [3]:
import pandas as pd

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity

layout1 = pd.read_csv('F:/Entity_Matching/pythonProject/data/ABC_layout_1.csv')
layout2 = pd.read_csv('F:/Entity_Matching/pythonProject/data/PQR_layout_2.csv')
layout3 = pd.read_csv('F:/Entity_Matching/pythonProject/data/layout_3_voters.csv')
layout4 = pd.read_csv('F:/Entity_Matching/pythonProject/data/KLM_layout_4.csv')
layout5 = pd.read_csv('F:/Entity_Matching/pythonProject/data/PQR_layout_2.csv')

del layout1["Last Name"]
del layout2["Unnamed: 0"]
del layout4["Unnamed: 0"]

layout1 = layout1.rename(columns={"First Name": "Name", "Father Name": "Father_Name", "Permanent_Adress":"Permanent_Address"})
layout2 = layout2.rename(columns = {"Customer_ID": "Mobile Number"})
layout3 = layout3.rename(columns={"votersName": "Name", "votersFatherName": "Father_Name", "votersMotherName": "Mother Name", " Gender": "Gender", "Permanent_Adress":"Permanent_Address"})
layout4 = layout4.rename(columns={"Father Name": "Father_Name"})

layout1_ = layout1.copy()
layout2_ = layout2.copy()
layout3_ = layout3.copy()
layout4_ = layout4.copy()
layout5_ = layout5.copy()


def sanitize(df):
    return df.map(lambda x: x.replace(',', '').replace(' ', '').strip() if isinstance(x, str) else x)

layout1_ = sanitize(layout1_)
layout2_ = sanitize(layout2_)
layout3_ = sanitize(layout3_)
layout4_ = sanitize(layout4_)
layout5_ = sanitize(layout5_)

soup1 = ['Name', 'Date of Birth', 'Father_Name']
soup2 = ['Name', 'Date of Birth', 'Father_Name']
soup3 = ['Name', 'Date of Birth', 'Father_Name']
soup4 = ['Name', 'Date of Birth', 'Father_Name']
soup5 = ['Name', 'Date of Birth', 'Father_Name']

def create_soup(df, df_, soup, soup_name):
    df[soup_name] = df_[soup].apply(lambda x: ' '.join(x.values.astype(str)).lower(), axis=1)

create_soup(layout1, layout1_, soup1, 'soup1')
create_soup(layout2, layout2_, soup2, 'soup2')
create_soup(layout3, layout3_, soup3, 'soup3')
create_soup(layout4, layout4_, soup4, 'soup4')
create_soup(layout5, layout5_, soup5, 'soup5')


def column_remover(df):

    columns = ["Name", "Date of Birth", "Father_Name", "Temporary_Address", "Mobile Number", "Permanent_Address",  "Mother Name"]
    column_pairs = [(col, f"{col}_x", f"{col}_y") for col in columns]

    for new_col, col_x, col_y in column_pairs:
        if col_x in df.columns and col_y in df.columns:
            df[new_col] = df[col_x].combine_first(df[col_y])
            df.drop([col_x, col_y], axis=1, inplace=True)

    return df

def combine_layouts(A, B, soup_A, soup_B, threshold=0):
    tfidf = TfidfVectorizer(stop_words='english')
    
    combined_soup = pd.concat([A[soup_A], B[soup_B]], ignore_index=True)
    tfidf.fit(combined_soup)
    
    tfidf_matrix_A = tfidf.transform(A[soup_A])
    tfidf_matrix_B = tfidf.transform(B[soup_B])
    
    similarity = cosine_similarity(tfidf_matrix_A, tfidf_matrix_B)
    similarity_df = pd.DataFrame(similarity, index=A.index, columns=B.index)

    max_idx_row = similarity_df.idxmax(axis=1)
    similarity_mask = similarity_df.max(axis=1) > threshold
    
    combined_df = pd.DataFrame({
        soup_A: A[soup_A].values,
        soup_B: [B.loc[idx, soup_B] if mask else None for idx, mask in zip(max_idx_row.values, similarity_mask)]
    })
    
    result = pd.merge(pd.merge(A, combined_df, on=soup_A, how='left'), B, on=soup_B,  how='inner')
    result.drop(columns = soup_B, inplace = True)
    column_remover(result)
    return result

result_12 = combine_layouts(layout1, layout2, 'soup1', 'soup2')
result_123 = combine_layouts(result_12, layout3, 'soup1', 'soup3')
result_1234 = combine_layouts(result_123, layout4, 'soup1', 'soup4')
final_result = combine_layouts(result_1234, layout5, 'soup1', 'soup5')

In [4]:
final_result

,Customer Code,National Id,Temporary_Address,soup1,Gender_x,PAN_Number_x,votersID,votersAge,SpouseName,Gender_y,...,Customer ID,SC Number,Mobile Number,Unnamed: 0,Customer_ID,Gender,PAN_Number_y,Name,Date of Birth,Father_Name
0,21216874,AB123C,NaN,ramthapa 1990-01-01 rambahadurthapa,Male,ABCDE1234F,11116874,45,Sita Thapa,Male,...,3245,001.01.01,1234567890,0,1234567890,Male,ABCDE1234F,Ram Thapa,1990-01-01,Ram Bahadur Thapa
1,22359363,DE456F,"New Road, Pokhara, Nepal",sitashrestha 1991-02-02 hariprasadshrestha,Female,FGHIJ5678K,22259363,38,Ravi Sharma,Female,...,3246,001.01.02,2345678901,1,2345678901,Female,FGHIJ5678K,Sita Shrestha,1991-02-02,Hari Prasad Shrestha
2,33485241,GH789I,"Pulchowk, Lalitpur, Nepal",harigurung 1992-03-03 gopalkrishnagurung,Male,LMNOP9012L,33385241,52,Maya Adhikari,Male,...,3247,001.01.03,3456789012,2,3456789012,Male,LMNOP9012L,Hari Gurung,1992-03-03,Gopal Krishna Gurung
3,45475489,JK012L,"Bagbazar, Kathmandu, Nepal",gitatamang 1993-04-04 shyamlaltamang,Female,QRSTU3456M,44475489,30,Surya Rai,Female,...,3248,001.01.04,4567890123,3,4567890123,Female,QRSTU3456M,Gita Tamang,1993-04-04,Shyam Lal Tamang
4,56562139,MN345O,NaN,mohanlama 1994-05-05 krishnarajlama,Male,VWXYZ7890N,55562139,27,Sarita Karki,Male,...,3249,001.01.05,5678901234,4,5678901234,Male,VWXYZ7890N,Mohan Lama,1994-05-05,Krishna Raj Lama
5,67698214,PQ678R,"Chabahil, Kathmandu, Nepal",radhamagar 1995-06-06 narayankumarmagar,Female,ABCD1234PQ,66698214,41,Pemba Gurung,Female,...,3250,001.01.06,6789012345,5,6789012345,Female,ABCD1234PQ,Radha Magar,1995-06-06,Narayan Kumar Magar
6,78714635,ST901U,"Kumaripati, Lalitpur, Nepal",krishnarai 1996-07-07 govindabahadurrai,Male,EFGH5678RS,77714635,36,Laxmi Shrestha,Male,...,3251,001.01.07,7890123456,6,7890123456,Male,EFGH5678RS,Krishna Rai,1996-07-07,Govinda Bahadur Rai
7,89847326,VW234X,NaN,saritasherpa 1997-08-08 shivanarayansherpa,Female,TUVW9012XY,88847326,50,Raj Maharjan,Female,...,3252,001.01.08,8901234567,7,8901234567,Female,TUVW9012XY,Sarita Sherpa,1997-08-08,Shiva Narayan Sherpa
8,91953421,YZ567A,"Boudha, Kathmandu, Nepal",bikashkarki 1998-09-09 bhagirathbahadurkarki,Male,ZABC3456DE,99953421,43,Pema Tamang,Male,...,3253,001.01.09,9012345678,8,9012345678,Male,ZABC3456DE,Bikash Karki,1998-09-09,Bhagirath Bahadur Karki
9,10203847,BC890D,"Kalanki, Kathmandu, Nepal",nishabhandari 1999-10-10 suryabahadurbhandari,Female,FGHI7890JK,10103847,34,Manish KC,Female,...,3254,001.01.10,123456789,9,123456789,Female,FGHI7890JK,Nisha Bhandari,1999-10-10,Surya Bahadur Bhandari
